## Searching for bouts for a day of ephys recording
- mircophone chan and sync data and bouts are extracted in sglx_preprocess-s_b1253_21-multisess-pouli-ephys

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2022-03-28 19:48:22,779 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

### Get the file locations for a session (day) of recordings

In [3]:
reload(et)
sess_par = {'bird': 's_b1267_22',
           'sess': '2022-03-28',
           'sort': 2}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='sglx')

raw_folder = exp_struct['folders']['sglx']

derived_folder = exp_struct['folders']['derived']

bouts_folder = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx')

sess_bouts_file = os.path.join(bouts_folder, 'bout_sess_auto.pickle')
sess_bouts_curated_file = os.path.join(bouts_folder, 'bout_curated.pickle')

#os.makedirs(bouts_folder, exist_ok=True)

In [4]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1267_22',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-28',
 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-28/sglx',
 'kwik': '/scratch/earneodo/s_b1267_22/sglx/kwik/2022-03-28',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1267_22/2022-03-28/sglx',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/sglx',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1267_22/sglx/msort/2022-03-28',
 'ksort': '/scratch/earneodo/s_b1267_22/sglx/ksort/2022-03-28/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/sglx/'}

### load concatenated the files of the session

In [5]:
def read_session_auto_bouts(exp_struct):
    # list all files of the session
    # read into list of pandas dataframes and concatenate
    # read the search parameters of the first session
    # return the big pd and the search params
    derived_folder = exp_struct['folders']['derived']
    
    search_params_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_search_params.pickle')
    hparams=None
    with open(search_params_files[0], 'rb') as fh:
        hparams = pickle.load(fh)
    
    sess_bout_pd_file = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx', 'bout_auto.pickle')
    try:
        bout_pd = pd.read_pickle(sess_bout_pd_file)
    except FileNotFoundError:
        logger.info('Not found session pandas file {}, loading each epoch file and concatenating'.format(sess_bout_pd_file))
        bout_pd_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_auto.pickle')
        logger.info('Files to load {}'.format(bout_pd_files))
        bout_pd = pd.concat([pd.read_pickle(p) for p in bout_pd_files[:]])
    
    bout_pd.reset_index(inplace=True, drop=True)
    return bout_pd, hparams

bout_pd, hparams = read_session_auto_bouts(exp_struct)

In [6]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      49 non-null     int64  
 1   end_ms        49 non-null     int64  
 2   start_sample  49 non-null     int64  
 3   end_sample    49 non-null     int64  
 4   p_step        49 non-null     object 
 5   rms_p         49 non-null     float64
 6   peak_p        49 non-null     float64
 7   bout_check    49 non-null     bool   
 8   file          49 non-null     object 
 9   len_ms        49 non-null     int64  
 10  syl_in        49 non-null     object 
 11  n_syl         49 non-null     int64  
 12  peaks_p       49 non-null     object 
 13  n_peaks       49 non-null     int64  
 14  l_p_ratio     49 non-null     float64
 15  waveform      49 non-null     object 
 16  confusing     49 non-null     bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 6.0+ KB


In [7]:
np.unique(bout_pd['start_ms']).shape

(49,)

In [8]:
### list all the source files
sess_wav_files = np.unique(bout_pd['file'].values)
sess_wav_files

array(['/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/sglx/1343_g0/wav_mic.wav'],
      dtype=object)

#### optional: drop the night recordings

In [9]:
# night_file = sess_wav_files[0]
# bout_pd.drop(bout_pd[bout_pd['file']==night_file].index, inplace=True)
# print(night_file)

In [10]:
np.unique(bout_pd['file'].values)

array(['/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/sglx/1343_g0/wav_mic.wav'],
      dtype=object)

In [11]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      49 non-null     int64  
 1   end_ms        49 non-null     int64  
 2   start_sample  49 non-null     int64  
 3   end_sample    49 non-null     int64  
 4   p_step        49 non-null     object 
 5   rms_p         49 non-null     float64
 6   peak_p        49 non-null     float64
 7   bout_check    49 non-null     bool   
 8   file          49 non-null     object 
 9   len_ms        49 non-null     int64  
 10  syl_in        49 non-null     object 
 11  n_syl         49 non-null     int64  
 12  peaks_p       49 non-null     object 
 13  n_peaks       49 non-null     int64  
 14  l_p_ratio     49 non-null     float64
 15  waveform      49 non-null     object 
 16  confusing     49 non-null     bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 6.0+ KB


In [12]:
np.unique(bout_pd['start_ms']).size

49

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [13]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [14]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    ## fill in the epoch
    bout_pd['epoch'] = bout_pd['file'].apply(lambda x: et.split_path(x)[-2])
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)
    
    # set all to 'confusing' (unchecked) and 'bout_check' false (not a bout)
    bout_pd['confusing'] = True
    bout_pd['bout_check'] = False

cleanup(bout_pd)

In [15]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        49 non-null     int64  
 1   end_ms          49 non-null     int64  
 2   start_sample    49 non-null     int64  
 3   end_sample      49 non-null     int64  
 4   p_step          49 non-null     object 
 5   rms_p           49 non-null     float64
 6   peak_p          49 non-null     float64
 7   bout_check      49 non-null     bool   
 8   file            49 non-null     object 
 9   len_ms          49 non-null     int64  
 10  syl_in          49 non-null     object 
 11  n_syl           49 non-null     int64  
 12  peaks_p         49 non-null     object 
 13  n_peaks         49 non-null     int64  
 14  l_p_ratio       49 non-null     float64
 15  waveform        49 non-null     object 
 16  confusing       49 non-null     bool   
 17  valid_waveform  49 non-null     bool 

In [16]:
np.unique(bout_pd['start_ms']).shape

(49,)

In [17]:
bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch
0,188965,194205,7558600,7768200,"[24.442159175867143, 28.28174389399953, 26.126...",4.717098,213.794822,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5240,"[[188965, 189105], [189740, 190285], [190730, ...",13,"[1, 8, 16, 25, 144, 164, 173, 181, 198, 210, 2...",32,163.750000,"[-13, -15, 6, -4, -28, -11, -13, -6, -4, 10, 1...",True,True,True,1343_g0
1,282685,287705,11307400,11508200,"[14.769791372603606, 20.159457531037955, 21.91...",4.717098,26.046679,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5020,"[[282685, 282805], [284750, 284885], [286560, ...",4,"[2, 12, 19, 415, 423, 435, 731, 778, 787, 796,...",15,334.666667,"[-13, -6, -8, -10, -11, 7, -2, 8, -15, 2, 8, 1...",True,True,True,1343_g0
2,531450,537625,21258000,21505000,"[3.8552895398396787, 93.4522004783155, 46.7810...",4.717098,127.416307,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,6175,"[[531450, 531495], [531595, 531630], [531695, ...",10,"[1, 20, 33, 49, 57, 80, 92, 122, 133, 151, 174...",32,192.968750,"[15, 34, 29, 16, 31, 25, 37, 23, 28, 34, 38, 4...",True,True,True,1343_g0
3,914655,923645,36586200,36945800,"[14.84304191265322, 81.90341068147794, 53.9810...",4.717098,81.903411,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,8990,"[[914655, 914715], [915365, 915410], [915705, ...",27,"[1, 17, 106, 142, 149, 205, 217, 234, 251, 258...",83,108.313253,"[-4, -52, -23, -14, -41, -23, -23, -12, -18, -...",True,True,True,1343_g0
4,934380,941160,37375200,37646400,"[65.47028488109002, 46.82532277779739, 12.5967...",4.717098,247.822719,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,6780,"[[934380, 934430], [935570, 935605], [936680, ...",12,"[4, 58, 170, 214, 224, 243, 270, 428, 450, 461...",34,199.411765,"[-24, -30, -26, -34, -34, -34, -26, -36, -36, ...",True,True,True,1343_g0
5,975775,983815,39031000,39352600,"[8.881195391993051, 9.017650519488793, 4.22163...",4.717098,86.742100,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,8040,"[[975775, 975815], [975945, 975980], [976125, ...",19,"[1, 35, 70, 88, 117, 156, 170, 187, 303, 320, ...",52,154.615385,"[149, 107, 152, 103, 114, 125, 122, 119, 109, ...",True,True,True,1343_g0
6,1136775,1143740,45471000,45749600,"[23.139207296411453, 15.450130857377205, 13.38...",4.717098,101.408415,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,6965,"[[1136775, 1136810], [1137015, 1137045], [1137...",15,"[48, 60, 79, 110, 118, 136, 150, 174, 187, 194...",45,154.777778,"[-99, -95, -91, -92, -84, -86, -92, -84, -90, ...",True,True,True,1343_g0
7,1196090,1201885,47843600,48075400,"[5.715657473272793, 9.381177383918274, 17.5613...",4.717098,233.267136,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5795,"[[1196090, 1196160], [1196895, 1196980], [1197...",11,"[3, 13, 162, 169, 176, 192, 203, 213, 220, 227...",46,125.978261,"[12, 13, 22, 9, 1, -16, -21, -34, -2, -16, -22...",True,True,True,1343_g0
8,1227855,1234175,49114200,49367000,"[1.279244872474135, 8.981778830217072, 13.3337...",4.717098,73.369345,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,6320,"[[1227855, 1227885], [1229310, 1229365], [1229...",10,"[4, 291, 302, 313, 327, 341, 382, 393, 424, 51...",34,185.882353,"[28, 21, -4, 6, -13, 15, 15, 12, 2, 12, 3, 9, ...",True,True,True,1343_g0
9,1525065,1530815,61002600,61232600,"[52.31580648331195, 32.845332023892155, 40.899...",4.717098,93.773680,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5750,"[[1525065, 1525105], [1526650, 1526680], [1526...",15,"[5, 16, 54, 121, 132, 216, 227, 234, 269, 295,...",51,112.745098,"[52, 60, 58, 56, 68, 48, 56, 24, 55, 65, 42, 3...",True,True,True,1343_g0


#### compute the spectrograms

In [18]:
# but don't compute if they were already computed
# try:
#     bout_pd = pd.read_pickle(sess_bouts_file) ## make it jump if there is no sess_bouts file
#     one_spec = bout_pd['spectrogram'][0] ## make it jump if there is no spectrogram key
#     logger.info('loaded bout pandas with spectrograms from ' + sess_bouts_file)
# except KeyError:
#     logger.info('Computing spectrograms')
#     bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
#     logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
#     bout_pd.to_pickle(sess_bouts_file)

In [19]:
# just compute them alright?
logger.info('Computing spectrograms')
bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
bout_pd.to_pickle(sess_bouts_file)

2022-03-28 19:48:24,450 root         INFO     Computing spectrograms
2022-03-28 19:48:31,932 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/bouts_sglx/bout_sess_auto.pickle


In [20]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,188965,194205,7558600,7768200,"[24.442159175867143, 28.28174389399953, 26.126...",4.717098,213.794822,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5240,...,13,"[1, 8, 16, 25, 144, 164, 173, 181, 198, 210, 2...",32,163.750000,"[-13, -15, 6, -4, -28, -11, -13, -6, -4, 10, 1...",True,True,True,1343_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,282685,287705,11307400,11508200,"[14.769791372603606, 20.159457531037955, 21.91...",4.717098,26.046679,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5020,...,4,"[2, 12, 19, 415, 423, 435, 731, 778, 787, 796,...",15,334.666667,"[-13, -6, -8, -10, -11, 7, -2, 8, -15, 2, 8, 1...",True,True,True,1343_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [21]:
bout_pd['file'][0]

'/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/sglx/1343_g0/wav_mic.wav'

## inspect the bouts and curate them

#### visualize one bout

In [22]:
bout_pd.iloc[0]

start_ms                                                     188965
end_ms                                                       194205
start_sample                                                7558600
end_sample                                                  7768200
p_step            [24.442159175867143, 28.28174389399953, 26.126...
rms_p                                                      4.717098
peak_p                                                   213.794822
bout_check                                                    False
file              /mnt/sphere/speech_bci/derived_data/s_b1267_22...
len_ms                                                         5240
syl_in            [[188965, 189105], [189740, 190285], [190730, ...
n_syl                                                            13
peaks_p           [1, 8, 16, 25, 144, 164, 173, 181, 198, 210, 2...
n_peaks                                                          32
l_p_ratio                                       

In [23]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [24]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bout_pd.iloc[24])

    'data': [{'type': 'scatter',
              'uid': '0284a8a3-5e4a-4fb2-8aff-a…

In [25]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,188965,194205,7558600,7768200,"[24.442159175867143, 28.28174389399953, 26.126...",4.717098,213.794822,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5240,...,13,"[1, 8, 16, 25, 144, 164, 173, 181, 198, 210, 2...",32,163.750000,"[-13, -15, 6, -4, -28, -11, -13, -6, -4, 10, 1...",True,True,True,1343_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,282685,287705,11307400,11508200,"[14.769791372603606, 20.159457531037955, 21.91...",4.717098,26.046679,False,/mnt/sphere/speech_bci/derived_data/s_b1267_22...,5020,...,4,"[2, 12, 19, 415, 423, 435, 731, 778, 787, 796,...",15,334.666667,"[-13, -6, -8, -10, -11, 7, -2, 8, -15, 2, 8, 1...",True,True,True,1343_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [26]:
bout_pd.reset_index(drop=True, inplace=True)

In [27]:
## Set confusing by default, will only be False once asserted bout/or not
bout_pd['confusing'] = True
bout_pd['bout_check'] = False

In [28]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [29]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bout_pd)

In [30]:
raise KeyboardInterrupt

KeyboardInterrupt: 

### save it

In [31]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': 'read_wav_chan',
 'file_order_fun': 'sess_file_id',
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle',
 'sample_rate': 40000}

In [32]:
### get the curated file path
##save to the curated file path
viz_bout.bouts_pd.to_pickle(sess_bouts_curated_file)
logger.info('saved curated bout pandas to pickle {}'.format(sess_bouts_curated_file))

2022-03-28 19:50:35,280 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/bouts_sglx/bout_curated.pickle


In [ ]:
sb.save_auto_bouts(viz_bout.bouts_pd, sess_par, hparams)

In [ ]:
viz_bout.bouts_pd

In [ ]:
### some by hand looking at the generated wav file 
#/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-28/sglx/1343_g0/wav_mic.wav

[[549.7, 552.1],
 [1029.85, 1030.86],
 []

### summary of where the good bouts are along the session

In [ ]:
def give_summary(bpd: pd.DataFrame):
    all_files = np.unique(bpd['file'])
    summary_dict = {f: np.where((bpd['bout_check']==True) & (bpd['file']==f))[0].size for f in all_files}
    return summary_dict

sum_dict = give_summary(viz_bout.bouts_pd)
sum_dict

## Some helper functions for somewhere down the line

In [ ]:
sess_par_e = dict.copy(sess_par)
sess_par_e['epoch'] = '0610_g0' 

In [ ]:
bpd.head()

In [ ]:
reload(et)
bpd = et.get_epoch_bout_pd(sess_par_e, only_curated=True)
bpd.head()

In [ ]:
bpd.head(1)

In [ ]:
bpd['start_ms'].values

In [ ]:
bpd['end_ms'].values - bpd['start_ms'].values

In [ ]:
bpd